In [4]:
# Read eidCodesMap from json and liver feats

import json
import pickle

work_loc = '/run/media/anton/Elements/UKB'

liver_feats = pickle.load(open(f'{work_loc}/liver_feats10-15.pkl', 'rb'))
subs = sorted(list(liver_feats.keys()))

# Get number of codes

with open(f'{work_loc}/liverCodes/codes.json', 'r') as f:
    codes = json.load(f)
    D = len(codes)
    print(D)

with open(f'{work_loc}/liverCodes/eidCodesMap.json', 'r') as f:
    eidCodesMap = json.load(f)

print(len(eidCodesMap))

for k in eidCodesMap:
    print(eidCodesMap[k])
    break

kept_feats = dict()
kept_subs = []

for k in subs:
    if k in eidCodesMap:
        kept_subs.append(k)
        kept_feats[k] = liver_feats[k]

print(len(subs))
print(len(kept_subs))
print(len(kept_feats))

1291
8564
[[888, 1278]]
10023
8564
8564


In [2]:
# Write liver subs to text file

with open(f'{work_loc}/liversubs.txt', 'w') as f:
    for sub in subs:
        f.write(sub)
        f.write('\n')

print('Done')

Done


In [5]:
import numpy as np

# Prediction from liver features

xliver = []
yliver = []

for k in kept_subs:
    v = eidCodesMap[k]
    xone = kept_feats[k]
    yone = np.zeros(D)
    if len(v) < 1:
        continue
    for mycodes in v:
        if mycodes is None:
            continue
        for code in mycodes:
            if code is not None:
                yone[code] = 1
    xliver.append(xone)
    yliver.append(yone)

xliver = np.stack(xliver)
yliver = np.stack(yliver)

print(xliver.shape)
print(yliver.shape)

(8564, 10)
(8564, 1291)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

xtr, xt, ytr, yt = train_test_split(xliver, yliver, train_size=3000)

for i in range(ytr.shape[1]):
    ytri = ytr[:,i]
    yti = yt[:,i]
    if not ytri.any() or not yti.any():
        continue
    reg = LogisticRegression(C=10000, max_iter=1000).fit(xtr, ytri)
    yhat = reg.predict(xt)
    tn, fp, fn, tp = confusion_matrix(yti, yhat).ravel()
    sp = tn / (tn+fp)
    sn = tp / (tp+fn)
    print(i, codes[i], sp, sn)

1 A04 1.0 0.0
4 A08 0.9994598487576521 0.0
5 A09 0.9998141609366289 0.0
10 A39 0.9983818770226537 0.0
11 A40 0.9992804461234035 0.0
12 A41 1.0 0.0
14 A46 1.0 0.0
15 A48 1.0 0.0
16 A49 0.9996396396396396 0.0
18 A63 1.0 0.0
23 A87 1.0 0.0
26 B00 1.0 0.0
27 B01 0.9976631314039187 0.0
28 B02 1.0 0.0
29 B07 0.9994591671173607 0.0
33 B17 1.0 0.0
34 B18 0.9987403275148461 0.0
37 B24 1.0 0.0
39 B34 0.99945779866257 0.0
40 B35 0.998920474991004 0.0
41 B36 0.9994607226316735 0.0
42 B37 1.0 0.0
44 B44 0.9992808342322905 0.0
49 B58 1.0 0.0
51 B80 0.9991008811364862 0.0
52 B82 0.9973031283710896 0.0
56 B94 0.9996404817544491 0.0
57 B95 1.0 0.0
58 B96 0.9998163115356355 0.0
59 B97 1.0 0.0
60 B98 0.999818676337262 0.0
61 B99 0.9996402877697842 0.0
63 C01 0.9992805755395684 0.0
64 C02 1.0 0.0
65 C07 0.9996403524545945 0.0
66 C09 1.0 0.0
71 C16 0.999280704909189 0.0
73 C18 1.0 0.0
74 C19 0.9994599459945994 0.0
75 C20 1.0 0.0
76 C21 0.9987414599065084 0.0
77 C22 1.0 0.0
79 C25 0.9994598487576521 0.0
80 